# 💬 TMCD – Trabalho Final
## Análise de Sentimentos em Reviews de Filmes

### 👥 Grupo Trab-grupo-30
- **Rafael Alexandre Dias Andorinha**, nº 131000  
- **Pedro Fonte Santa**, nº 105306  

---

📅 **Data de entrega:** 26 de abril  

📊 **Objetivo deste script:** Este Notebook corresponde à Tarefa 2.6 do trabalho.

O objetivo principal desta tarefa é aplicar modelos generativos ao problema de análise de sentimentos, com foco na classificação automática de reviews em português como `pos` (positivo) ou `neg` (negativo).

Foram testados três modelos distintos:

- Um modelo pré-treinado específico para sentimento multilingue (`cardiffnlp/twitter-xlm-roberta-base-sentiment`);
- Um modelo baseado em classificação por estrelas (`nlptown/bert-base-multilingual-uncased-sentiment`);
- E uma abordagem zero-shot com base em inferência textual (`facebook/bart-large-mnli`).

Através da análise de desempenho no conjunto `notebook-devel.csv`, concluiu-se que o modelo `nlptown` apresentou os melhores resultados.

Com base nesta avaliação, optámos por utilizar o modelo `nlptown/bert-base-multilingual-uncased-sentiment` para classificar automaticamente as 421 reviews do ficheiro `notebook-challenge.txt`.

O resultado final foi exportado no formato `notebook-challenge.csv`, respeitando a estrutura esperada (`sentiment` e `review_text` separados por tabulação).

---

# 🗂️ Dataset: IMDB Reviews

In [ ]:
!pip install transformers datasets evaluate --quiet


In [4]:
import pandas as pd
import numpy as np
from transformers import pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [26]:
# Ajusta o caminho se for diferente no teu Drive
devel_path = "/content/drive/MyDrive/ISCTE/Mestrado/TM/projeto/dataset/notebooks-devel.csv"

df_devel = pd.read_csv(devel_path, sep="\t")
df_devel.head()

,sentiment,review_text
0,pos,Estou contente com a compra entrega rápida o ú...
1,pos,"Excelente!!! Produto muito bom, bonito, design..."
2,pos,"Superou expectativas É excelente para jogos, ..."
3,pos,Comprei o mesmo para minha irmã e ela me conto...
4,pos,Notebbok muito bom! Rápido pequeno e leve para...


In [27]:
# Verificar valores únicos
print(df_devel['sentiment'].value_counts())


sentiment
pos    1205
neg     295
Name: count, dtype: int64


### 🤖 Exp. 1 - Carregar modelo e pipeline - cardiffnlp

In [11]:
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Criar pipeline
clf = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Device set to use cuda:0
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [30]:
# Aplicar o modelo ao ficheiro completo
preds = clf(df_devel['review_text'].tolist(), truncation=True)

# Converter output para 'pos' / 'neg'
def map_sentiment(pred):
    label = pred['label'].lower()
    if label == 'positive':
        return 'pos'
    elif label == 'negative':
        return 'neg'
    else:
        # Ignorar 'neutral' ou mapear para o mais próximo
        return 'pos'  # ou 'neg'

df_devel['pred'] = [map_sentiment(p) for p in preds]

In [31]:
print("📊 Avaliação do modelo no notebook-devel.csv (amostra)")
print(classification_report(df_devel['sentiment'], df_devel['pred'], digits=3))


📊 Avaliação do modelo no notebook-devel.csv (amostra)
              precision    recall  f1-score   support

         neg      0.807     0.881     0.843       295
         pos      0.970     0.949     0.959      1205

    accuracy                          0.935      1500
   macro avg      0.889     0.915     0.901      1500
weighted avg      0.938     0.935     0.936      1500



In [23]:
# Aplicar o modelo ao ficheiro inteiro
preds = clf(df_devel['review_text'].tolist(), truncation=True)

In [24]:
def map_sentiment(pred):
    label = pred['label'].lower()
    if label == 'positive':
        return 'pos'
    elif label == 'negative':
        return 'neg'
    else:
        return 'pos'  # Força neutral para positivo (justificado pelo teu teste anterior)

# Aplicar o mapeamento
df_devel['pred'] = [map_sentiment(p) for p in preds]


In [25]:
print("📊 Avaliação do modelo no notebook-devel.csv (completo)")
print(classification_report(df_devel['sentiment'], df_devel['pred'], digits=3))

📊 Avaliação do modelo no notebook-devel.csv (completo)
              precision    recall  f1-score   support

         neg      0.807     0.881     0.843       295
         pos      0.970     0.949     0.959      1205

    accuracy                          0.935      1500
   macro avg      0.889     0.915     0.901      1500
weighted avg      0.938     0.935     0.936      1500



### 🤖 Exp. 2 - Carregar modelo e pipeline - nlptown

In [35]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
clf = pipeline("text-classification", model=model_name, tokenizer=model_name)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [36]:
# Fazer as previsões
preds = clf(df_devel['review_text'].tolist(), truncation=True)

# Função para mapear estrelas para pos/neg
def map_sentiment_stars(pred):
    label = pred['label']
    if label in ['1 star', '2 stars']:
        return 'neg'
    elif label in ['4 stars', '5 stars']:
        return 'pos'
    else:
        # Se for '3 stars', podemos assumir positivo ou neutro, depende do que quiseres
        return 'pos'  # Aqui para otimizar positivo

df_devel['pred'] = [map_sentiment_stars(p) for p in preds]


In [37]:
print("📊 Avaliação do novo modelo no notebook-devel.csv")
print(classification_report(df_devel['sentiment'], df_devel['pred'], digits=3))


📊 Avaliação do novo modelo no notebook-devel.csv
              precision    recall  f1-score   support

         neg      0.823     0.915     0.867       295
         pos      0.979     0.952     0.965      1205

    accuracy                          0.945      1500
   macro avg      0.901     0.934     0.916      1500
weighted avg      0.948     0.945     0.946      1500



### 🤖 Exp. 3 - Carregar modelo e pipeline - facebook/bart-large-mnli

In [44]:
from transformers import pipeline

zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cuda:0


In [42]:
# Candidatos a classificação binária
labels = ["positivo", "negativo"]

# Função que aplica o zero-shot classification
def classify_zero_shot(text):
    result = zero_shot(text, candidate_labels=labels, hypothesis_template="O sentimento deste texto é {}.")
    return "pos" if result['labels'][0] == "positivo" else "neg"

# Lista de textos
texts = df_devel['review_text'].tolist()

# Aplicar o modelo em batch
results = zero_shot(
    texts,
    candidate_labels=["positivo", "negativo"],
    hypothesis_template="O sentimento deste texto é {}.",
    batch_size=8
)

# Extrair a melhor label para cada resultado
df_devel['pred'] = ['pos' if r['labels'][0] == 'positivo' else 'neg' for r in results]


In [43]:
print("📊 Avaliação do modelo zero-shot com BART-MNLI")
print(classification_report(df_devel['sentiment'], df_devel['pred'], digits=3))


📊 Avaliação do modelo zero-shot com BART-MNLI
              precision    recall  f1-score   support

         neg      0.623     0.864     0.724       295
         pos      0.963     0.872     0.916      1205

    accuracy                          0.871      1500
   macro avg      0.793     0.868     0.820      1500
weighted avg      0.896     0.871     0.878      1500



### 📋 Tabela Comparativa das 3 Experiências

| Experiência | Modelo Utilizado                                 | Accuracy | F1-Score (neg) | F1-Score (pos) |
|-------------|--------------------------------------------------|----------|----------------|----------------|
| Exp. 1      | cardiffnlp/twitter-xlm-roberta-base-sentiment    | 0.935    | 0.843          | 0.959          |
| Exp. 2      | nlptown/bert-base-multilingual-uncased-sentiment | 0.945    | 0.867          | 0.965          |
| Exp. 3      | facebook/bart-large-mnli (zero-shot)             | 0.871    | 0.724          | 0.916          |


### Com base neste resultados vamos decidir usar o modelo nlptown para gerar o csv **notebook-challenge.csv**

### 📦 Gerar o notebook-challenge.csv

In [12]:
challenge_path = "/content/drive/MyDrive/ISCTE/Mestrado/TM/projeto/dataset/notebooks-challenge.txt"


# Carregar o ficheiro
with open(challenge_path, "r", encoding="utf-8") as f:
    reviews_challenge = f.read().splitlines()

# Ignorar a primeira linha ("review_text")
reviews_challenge = reviews_challenge[1:]

# Verificar
print(reviews_challenge[:5])


['O note é bom, mas não superou as expectativas quanto a velocidade, por isso sugiro 8GB', 'Ele corresponde ao valor pago. Custando em média R$ 2000, não encontraremos nada melhor.', 'O notebook é muito bom, o Windows 10 ferra um pouco com a velocidade, mas isso é coisa do sistema operacional. Recomendo muito esse produto.', 'O notebook tem uma tela enorme, muito útil para trabalhar e fazer trabalhos acadêmicos. É lindo...', 'Muito bom ! Excelente notebook! Entrega rapida. Fiquei muito satisfeita.']


In [ ]:
from transformers import pipeline

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
clf = pipeline("text-classification", model=model_name, tokenizer=model_name)

In [13]:
# Fazer as previsões
preds_challenge = clf(reviews_challenge, truncation=True)

# Função para mapear estrelas para pos/neg
def map_sentiment_stars(pred):
    label = pred['label']
    if label in ['1 star', '2 stars']:
        return 'neg'
    elif label in ['4 stars', '5 stars']:
        return 'pos'
    else:
        return 'pos'  # Assume 3 estrelas como positivo (para ser mais otimista)

# Aplicar o mapeamento
sentiments_challenge = [map_sentiment_stars(p) for p in preds_challenge]


In [14]:
# Analisar distribuição das classes previstas
pd.Series(sentiments_challenge).value_counts()

,count
pos,312
neg,109


In [22]:
# Criar o dataframe
df_challenge = pd.DataFrame({
    "sentiment": sentiments_challenge,
    "review_text": reviews_challenge
})

# Verificar as primeiras linhas
df_challenge.head()

,sentiment,review_text
0,pos,"O note é bom, mas não superou as expectativas ..."
1,pos,Ele corresponde ao valor pago. Custando em méd...
2,pos,"O notebook é muito bom, o Windows 10 ferra um ..."
3,pos,"O notebook tem uma tela enorme, muito útil par..."
4,pos,Muito bom ! Excelente notebook! Entrega rapida...


In [21]:
import csv

# Exportar para CSV com separador TAB e sem aspas
output_path = "/content/drive/MyDrive/ISCTE/Mestrado/TM/projeto/dataset/notebook-challenge.csv"

df_challenge.to_csv(
    output_path,
    sep="\t",
    index=False,
    quoting=csv.QUOTE_NONE,
    escapechar="\\"
)

print(f"✅ Ficheiro exportado para: {output_path}")

✅ Ficheiro exportado para: /content/drive/MyDrive/ISCTE/Mestrado/TM/projeto/notebook-challenge.csv
